In [16]:
#gerekli kütüphanelerimizi (paketleri) import tanımlıyoruz
import matplotlib.pyplot as plt
import glob 
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D #(katman,seyreltme,düzleştirme,evrişim ağı,piksel ekleme)
from PIL import Image
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import sklearn
"""  https://trex-runner.com/  """
#from sklearn.preprocessing import train_test_split
from sklearn.model_selection import train_test_split
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")

imgs = glob.glob("./img/*.png")  # *'ın anlamı --> ismi ne olursa olsun png formatındaki tüm fotoğrafları al

width = 125
height = 50
X = []  #resimler ve etiketler için boş bir liste oluşturuyoruz
Y = []

for img in imgs: #resimler içinde geziniyoruz
    filename = os.path.basename(img) #img isimlerinde geziniyoruz
    label = filename.split("_")[0]  # ['down','a4e31f8e-42f3-4560-ac6c-6f56884b7b30','72'] --> down

    im = np.array(Image.open(img).convert('L').resize((width,height))) #resimlerimizi açıp yeniden boyutlandırıp im değişkenine atıyoruz
    im = im/255 #normalize ediyorum yani 0-1 aralığına sokuyorum 

    X.append(im) 
    Y.append(label)

X = np.array(X) # ' train_test_split ' methodu içine array aldığından array'e çevirdik
X = X.reshape(X.shape[0],width,height,1) # 'X.shape[0]' kaç resim olduğunun sayısı 1 ise channel (kanal)

# label_encoder = LabelEncoder()
# Y = label_encoder.fit_transform(Y)
# sns.countplot(Y)

def onehot_labels(values):
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)   # [0,0,0,0,0,0,0,0,0,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2] tarzına çeviriyor
    onehot_encoded = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded),1)  
    onehot_encoded = onehot_encoded.fit_transform(integer_encoded) # 0 --> [1.,0.,0.]        1 --> [0.,1.,0.,]      2--> [0.,0.,1.]
    return onehot_encoded

Y = onehot_labels(Y)

train_x, test_x, train_y, test_y  =  train_test_split(X,Y,test_size=0.25,random_state=2) #test_size = 0.25 dersek 0.75 i eğitim için ayrılır  , 

#CNN Model
model = Sequential()  # 'Sequential' layerları (katmanları) üzerine ekliyeceğimiz temel yapı 
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape = (width,height,1) ))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25)) # yüzde 25'i kaybolsun yüzde 75'i ile ilgilenelim
model.add(Flatten()) #düzleştirdim çünkü sınıflandıracağım
model.add(Dense(128,activation='relu')) #128 nöron
model.add(Dropout(0.4)) 
model.add(Dense(3,activation='softmax')) #çıktı layerimiz (softmaxi ikiden fazla çıktımız olduğunda kullanıyoruz)

# if os.path.exists("./trex_weight.h5"):
#     model.load_weights('trex_weight.h5')
#     print('Weights Yuklendi')

model.compile(loss = 'categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(train_x, train_y, epochs=35, batch_size=64) # epochs = resimlerimizin toplamda kaç kez eğitileceği, batch_size = resimlerimizin kaç grup halinde iterasyona sokulacağı

score_train = model.evaluate(train_x,train_y)
score_test = model.evaluate(test_x,test_y)

print('Loss (kayıp) : %',score_train[0]*100)
print('Accuary (Eğitim Doğruluğu) : %',score_train[1]*100)
print('Accuary (Test Doğruluğu) : %',score_test[1]*100)

open('model.json','w').write(model.to_json())  #modeli kaydediyoruz
model.save_weights('trex_weight.h5')  




Epoch 1/35
2/2 [==============================] - 5s 1s/step - loss: 3.2698 - accuracy: 0.4831
Epoch 2/35
2/2 [==============================] - 3s 1s/step - loss: 3.2193 - accuracy: 0.6186
Epoch 3/35
2/2 [==============================] - 2s 1s/step - loss: 1.5343 - accuracy: 0.6356
Epoch 4/35
2/2 [==============================] - 2s 955ms/step - loss: 0.7705 - accuracy: 0.7542
Epoch 5/35
2/2 [==============================] - 2s 980ms/step - loss: 0.4522 - accuracy: 0.7966
Epoch 6/35
2/2 [==============================] - 2s 934ms/step - loss: 0.3779 - accuracy: 0.8305
Epoch 7/35
2/2 [==============================] - 2s 1s/step - loss: 0.3097 - accuracy: 0.8814
Epoch 8/35
2/2 [==============================] - 2s 940ms/step - loss: 0.2837 - accuracy: 0.9492
Epoch 9/35
2/2 [==============================] - 2s 995ms/step - loss: 0.2141 - accuracy: 0.9407
Epoch 10/35
2/2 [==============================] - 2s 943ms/step - loss: 0.2082 - accuracy: 0.9407
Epoch 11/35
2/2 [==============